<a href="https://colab.research.google.com/github/Go21032/object_detection_golib/blob/main/%E3%82%AB%E3%83%A1%E3%83%A9%E3%81%A7%E3%83%86%E3%82%B9%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 2.0系tensorflow選択
#%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
# 必要なライブラリをインポート
# time：　時間を扱うモジュール ⇨ http://ja.pymotw.com/2/time/
import keras
import sys, os
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

#from keras.utils import np_utils  #20231217に使用できません．以下のコードを使用
from tensorflow.python.keras.utils import np_utils

from tensorflow.keras.preprocessing.image import img_to_array, load_img
import time

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# AIモデルファイルを自分のDriveからGoogle Driveからコピーする
# 「my_janken_best_model.h5」　は学習したモデルの名
!cp /content/drive/MyDrive/蓮ノ空CD機械学習/Hasu_best_model2.h5 './Hasu_best_model2.h5'

In [ ]:
# AIモデルファイルの情報確認
!ls -lh

total 1.1G
drwx------ 5 root root 4.0K Mar 28 00:11 drive
-rw------- 1 root root 1.1G Mar 28 00:11 Hasu_best_model2.h5
drwxr-xr-x 1 root root 4.0K Mar 26 13:28 sample_data


In [ ]:
#　最適モデルは自分が作ったモデルまたはVGG16_model_4の場合，これを実行：
# 「my_janken_best_model.h5」　は学習したモデルの名
keras_model = tf.keras.models.load_model("Hasu_best_model2.h5")
keras_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 192, 192, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 96, 96, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 294912)            0         
                                                                 
 dense_2 (Dense)             (None, 312)               92012856  
                                                                 
 dropout_1 (Dropout)         (None, 312)               0         
                                                                 
 dense_3 (Dense)             (None, 4)                 1252      
                                                      

In [ ]:
#　最適モデルはEfficientNetモデルより作ったものの場合，これを実行：
# 「my_janken_best_model.h5」　は学習したモデルの名
import tensorflow_hub as hub
keras_model = tf.keras.models.load_model('Hasu_best_model2.h5', custom_objects={'KerasLayer':hub.KerasLayer})
keras_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 192, 192, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 96, 96, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 294912)            0         
                                                                 
 dense_2 (Dense)             (None, 312)               92012856  
                                                                 
 dropout_1 (Dropout)         (None, 312)               0         
                                                                 
 dense_3 (Dense)             (None, 4)                 1252      
                                                      

In [ ]:
# 処理を行うコールバック関数を定義
# 画像認識の部分以外は標準コードである
# cv2: OpenCVによる画像処理に関するモジュール ⇨ https://oku.edu.mie-u.ac.jp/~okumura/python/opencv.html
# PIL: Pythonの画像処理ライブラリPillow(PIL) ⇨ https://note.nkmk.me/python-pillow-basic/
# BytesIOとは、メモリ上でバイナリデータを扱うための機能です。Python の標準ライブラリ io に含まれています。 ⇨ https://techacademy.jp/magazine/19185
# バイナリデータとは主に画像や音声などのデータのことです. BytesIO は、よく画像処理ライブラリ PIL(Pillow) と組み合わせて用いられ
# Base64：バイナリデータを一定の規則に基づき可読文字に置き換える変換方式の一つです.　⇨ https://docs.python.org/ja/3/library/base64.html
import IPython
from google.colab import output
from google.colab.patches import cv2_imshow # incompatible with Jupyter notebook
import cv2
import numpy as np
from PIL import Image, ImageDraw
from io import BytesIO
import base64

# 画像りサイズ（学習モデルと一致）
img_w, img_h = 192, 192

def run(img_str):
  #decode to image データの符号化，
  decimg = base64.b64decode(img_str.split(',')[1], validate=True)
  decimg = Image.open(BytesIO(decimg))
  decimg = np.array(decimg, dtype=np.uint8);
  decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

  # 学習したモデルを使って画像認識
  img = cv2.resize(decimg, (img_w, img_h))
  img = img_to_array(img)
  img = img / 255.0
  img = img[np.newaxis]
  img = np.asarray(img)
  preds = keras_model.predict(img)
  prelabel = np.argmax(preds, axis=1)
  if prelabel == 0: #if,elifの部分は具体的な問題に対して変更
    label='HoliHoli'
  elif prelabel == 1:
    label='Kabe'
  elif prelabel == 2:
    label='Natumeki'
  elif prelabel == 3:
    label='lttf'

  # 画像にテキストを書き込む ⇨ https://axa.biopapyrus.jp/ia/opencv/puttext.html
  # ⇨ https://shikaku-mafia.com/opencv-puttext/
  out_img = decimg
  cv2.putText(out_img, label, (10,80), cv2.FONT_HERSHEY_TRIPLEX, 2, (0, 255, 0), 3, cv2.LINE_AA)

  #encode to string
  _, encimg = cv2.imencode(".jpg", out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
  img_str = encimg.tobytes()
  img_str = ("data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8'))
  return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)

In [ ]:
# Webカメラを使うJavascriptのコード（標準コード） ⇨ 　https://qiita.com/a2kiti/items/f32de4f51a31d609e5a5　　  https://qiita.com/PoodleMaster/items/480b1da080282b15b584
from IPython.display import display, Javascript
from google.colab.output import eval_js

def use_cam(quality=0.8):
  js = Javascript('''
    async function useCam(quality) {
      const div = document.createElement('div');
      document.body.appendChild(div);
      //video element
      const video = document.createElement('video');
      video.style.display = 'None';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      //canvas for display. frame rate is depending on display size and jpeg quality.
      display_size = 540
      const src_canvas = document.createElement('canvas');
      src_canvas.width  = display_size;
      src_canvas.height = display_size * video.videoHeight / video.videoWidth;
      const src_canvasCtx = src_canvas.getContext('2d');
      src_canvasCtx.translate(src_canvas.width, 0);
      src_canvasCtx.scale(-1, 1);
      div.appendChild(src_canvas);

      const dst_canvas = document.createElement('canvas');
      dst_canvas.width  = src_canvas.width;
      dst_canvas.height = src_canvas.height;
      const dst_canvasCtx = dst_canvas.getContext('2d');
      div.appendChild(dst_canvas);

      //exit button
      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Exit';
      var exit_flg = true
      exit_btn.onclick = function() {exit_flg = false};
      btn_div.appendChild(exit_btn);

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      var send_num = 0
      // loop
      _canvasUpdate();
      async function _canvasUpdate() {
            src_canvasCtx.drawImage(video, 0, 0, video.videoWidth, video.videoHeight, 0, 0, src_canvas.width, src_canvas.height);
            if (send_num<1){
                send_num += 1
                const img = src_canvas.toDataURL('image/jpeg', quality);
                const result = google.colab.kernel.invokeFunction('notebook.run', [img], {});
                result.then(function(value) {
                    parse = JSON.parse(JSON.stringify(value))["data"]
                    parse = JSON.parse(JSON.stringify(parse))["application/json"]
                    parse = JSON.parse(JSON.stringify(parse))["img_str"]
                    var image = new Image()
                    image.src = parse;
                    image.onload = function(){dst_canvasCtx.drawImage(image, 0, 0)}
                    send_num -= 1
                })
            }
            if (exit_flg){
                requestAnimationFrame(_canvasUpdate);
            }else{
                stream.getVideoTracks()[0].stop();
            }
      };
    }
    ''')
  display(js)
  data = eval_js('useCam({})'.format(quality))

In [ ]:
use_cam()

<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 19ms/step
